## Recap

### Expressing class hierachies

- Case classes are Scala’s preferred way to define complex data

- As an example, we'll try to represent JSON with Scala's case class
  ```
  { 
    "firstName" : "John",
    "lastName" : "Smith",
    "address": {
      "streetAddress": "21 2nd Street",
      "state": "NY",
      "postalCode": 10021
    },
    "phoneNumbers": [
      {"type": "home", "number": "212 555-1234"},
      {"type": "fax", "number": "646 555-4567"}
    ]
  }
  ```

#### Using abstract class + companion object

- Think of the abstract class + companion object design pattern as akin to an `Enum`

- You have some abstract class (e.g. `Animal`), and you have some `case class` which are inheriting from `Animal` (e.g. LandAnimal, SeaAnimal)

- You don't want to pollute your namespace with these classes, but you want it to be clear how these are related. The abstract class + companion object pattern allows you to do exactly this

- The companion object also lets you define specific methods for the abstract class that you can use. This differs from defining an abstract method, where a method defined in the abstract class itself instead of the companion object must be implemented by all classes extending it

- Similarly, anything implmented directly in the abstract class is inherited by the subclasses extending it

  ```scala
  abstract class JSON
    def abstractMethod: Nil 

    val a: Int = 123 //inherited

  object JSON:
    case class Seq (elems: List[JSON]) extends JSON
    case class Obj (bindings: Map[String, JSON]) extends JSON
    case class Num (num: Double) extends JSON
    case class Str (str: String) extends JSON
    case class Bool(b: Boolean) extends JSON
    case object Null extends JSON

    def abstractClassMethod() = {
      println("call using JSON.abstractClassMethod()")
    }
  ```



#### Using enums

- In most cases, if all you want to represent is some class hierachy, then Enums are a more concise way of doing things
  
  ```scala
    enum JSON:
      case Seq (elems: List[JSON])
      case Obj (bindings: Map[String, JSON])
      case Num (num: Double)
      case Str (str: String)
      case Bool(b: Boolean)
      case Null

  val jsData = JSON.Obj(Map(
    "firstName" -> JSON.Str("John"),
    "lastName" -> JSON.Str("Smith"),
    "address" -> JSON.Obj(Map(
      "streetAddress" -> JSON.Str("21 2nd Street"),
      "state" -> JSON.Str("NY"),
      "postalCode" -> JSON.Num(10021)
    )),
    "phoneNumbers" -> JSON.Seq(List(
      JSON.Obj(Map(
        "type" -> JSON.Str("home"), 
        "number" -> JSON.Str("212 555-1234")
      )),
      JSON.Obj(Map(
        "type" -> JSON.Str("fax"), 
        "number" -> JSON.Str("646 555-4567")
      )) 
    )) 
  ))
  ```

#### Displaying JSON

- Using the `JSON` class hierachy, we can write the following method `show(json: JSON): String` that displays a json object

```scala
def inQuotes(str: String): String = {
  "\"" + str + "\""
}

def show(json: JSON): String = {
  json match
    case JSON.Seq(elems) =>
      elems.map(show).mkString("[", ", ", "]")
    case JSON.Obj(bindings) =>
      val assocs = bindings.map(
        (key, value) => s"${inQuotes(key)}: ${show(value)}"
      )
      assocs.mkString("{", ",\n ", "}")
    case JSON.Num(num) => num.toString
    case JSON.Str(str) => inQuotes(str)
    case JSON.Bool(b) => b.toString
    case JSON.Null => "null"
}
```

### Collections 

- Scala has a fixed hierachy built-ins to represent collections
  - Iterable
    - Seq
      - List
    - Set
    - Map

- Some core methods of all of these are
  - map
  - flatMap
  - filter
  - foldLeft/foldRight

```scala
  extension[T](xs: List[T])
    def map[U](f: T => U): List[U] = {
      xs match
        case head :: tail => {
          f(head) :: tail.map(f)
        }
        case Nil => Nil
    }

    def flatMap[U](f: T=>List[U]): List[U] = {
      xs match 
        case head :: tail => {
          f(head) ++ tail.flatMap(f)
        }
        case Nil => Nil
    }

    def filter(f: T => Boolean): List[T] = {
      xs match
        case head :: tail => {
          if f(head) then head :: tail.filter(f)
          else tail.filter(f)
        }
        case Nil => Nil
    }
```

### for expressions

- This is syntactic sugar. The 2 cases below are equal

```scala
  (1 to n)(i => 
    (1 until i)(j => 
      filter isPrime(i+j)
      map (i,j)
    )
  )

  for 
    i <- (1 to n)
    j <- (1 to i)
    if isPrime(i+j)
  yield
    (i,j)
```

- Applying this to the JSON class we defined earlier:

```scala
  def bindings(x: JSON): List[(String, JSON)] = {
    x match
      case JSON.Obj(bindings) => bindings.toList
      case _ => Nil
  }

  for 
    case ("phoneNumbers", JSON.Seq(numberInfos)) <- bindings(jsData)
    numberInfo <- numberInfos
    case ("number", JSON.Str(number)) <- bindings(numberInfo)
    if number.startsWith("212")
  yield
    number

```

## `for` Comprehension

- In Scala, the ability to match patterns using `for` loop is a really convenient pattern

- Let's imagine we have the following "database"

  ```scala
    val books: List[Book] = List(
      Book(
        title = "Structure and Interpretation of Computer Programs",
        authors = List("Abelson, Harald", "Sussman, Gerald J.")
      ),
      Book(
        title = "Introduction to Functional Programming",
        authors = List("Bird, Richard", "Wadler, Phil")
      ),
      Book(
        title = "Effective Java",
        authors = List("Bloch, Joshua")
      ),
      Book(
        title = "Java Puzzlers",
        authors = List("Bloch, Joshua", "Gafter, Neal")
      ),
      Book(
        title = "Programming in Scala",
        authors = List("Odersky, Martin", "Spoon, Lex", "Venners, Bill")
      )
    )
  ```

- Pattern matching allows us to use a `for` loop to generate the the Seq that we want. Suppose we want all titles where the author's name contains the letter `W`. Then
  ```scala
    for
      b <- books
      a <- book.authors
      if a.contains("W")
    yield
      b.title
  ```

- Let's do something a bit more complex. Find the names of all authors who have written at least two books present in the database.
  - See 2-slides.scala

## Translation of a `for`

- A `for` loop can be used to express many functions, including those that are treated as built ins
  
  ```scala
    def mapFun[T, U](xs: List[T], f: T => U): List[U] = {
      for
        x <- xs
      yield
        f(x)
    }

    def flatMap[T, U](xs: List[T], f: T => Iterable[U]): List[U] = {
      for 
        x <- xs 
        y <- f(x) 
      yield 
        y
    }

    def filter[T](xs: List[T], p: T => Boolean): List[T] = {
      for 
        x <- xs 
        if p(x) 
      yield 
        x
    }
  ```

- In fact, Scala's interpreter treats for loops as some of these higher order functions

```scala
  for
    i <- 1 until n
    j <- 1 until i
    if isPrime(i + j)
  yield (i, j)

  //equivalent to
  (1 until n).flatMap(i =>
    (1 until i)
      .withFilter(j => isPrime(i+j))
      .map(j => (i, j)))
```

### Exercise

- Translate the following expression

- See `3-slides.scala`

  ```scala
  for 
    b <- books 
    a <- b.authors 
    if a.startsWith("Bird")
  yield b.title
  ```

### Translation of `for` loops to `flatMap`, `map`, and `withFilter`

- This isn't just an arbitrary exercise. As long as the methods `flatMap`, `map`, and `withFilter` exist, Scala will be able to translate the loop into a `for` loop, regardless of type!

- So for example, if instead of `Books`, I have some database class that defines the 3 functions, I can use the `for` loop syntax to query the database. Super convenient!

- This is the basis for database connection frameworks like Spark

## Functional Random Generators

- The last point from the previous section deserves more attention. Basically a `for` expression can be applicable for **ANYTHING**, so long as it has an implementation of `map`, `flatMap`, and `withFilter`

- Let's take a look at a specific example of this: Generators

### Translating `for` in a generic `Generator` class

- Let's define a basic generator trait with an abstract `generate` method
  ```scala
    trait Generator[T+]:
      def generate(): T
  ```

- Using this trait, we can define some basic generators as follows
  ```scala
    // Integer generator
    val integerGenerator = new Generator[Int]: 
      val rand = java.util.Random()
      
      def generate(): Int = {
        rand.nextInt()
      }
    
    val booleanGenerator = new Generator[Boolean]:
      def generate(): Boolean = {
        integerGenerator.generate() > 0
      }

    val pairsGenerator = new Generator[(Int, Int)]:
      def generate(): (Int, Int) = {
        (integerGenerator.generate(), integerGenerator.generate())
      }
  ```

- However, notice that writing this creates a bunch of boilerplate. Can we simplify? Yes!
  ```scala
    // Simplified
    val booleanGenerator = integerGenerator.map(x => x > 0)

    val pairsGenerator = integerGenerator.flatMap(x => integerGenerator.map(y => (x,y)))
  ```

- This immediately runs into a problem; we have not yet defined the `map` and `flatMap` methods for the generator class! We can define the `map` and `flatMap` methods either via an `extension`, or directly within the `Generator` trait


- Note that `map` and `flatMap` should produce a new generator, NOT just mapping some function `f` to a generated value.

  ```scala
    trait Generator[+T]:
      def generate(): T

      // Viable as trait methods
      def map[S](f: T => S) = new Generator[S] {
        def generate(): S = {
          f(Generator.this.generate())
        }
      }

      def flatMap[S](f: T => Generator[S]) = new Generator[S]:
        def generate(): S = { 
          f(Generator.this.generate()).generate()
        }


    // Viable as an extension
    extension [T, S](g: Generator[T])
      def map(f: T => S) = new Generator[S]:
        def generate() = f(g.generate())
      
      def flatMap(f: T => Generator[S]) = new Generator[S]:
        def generate() = f(g.generate()).generate()
  ```

### Translating the booleanGenerator and pairGenerator objects

- With the implementation of `map` and `flatMap`, we can now implement the `booleanGenerator` and `pairGenerator` objects in a single line
  ```scala
    // Simplified
    val booleanGenerator = integerGenerator.map(x => x > 0)

    val pairsGenerator = integerGenerator.flatMap(x => integerGenerator.map(y => (x,y)))
  ```

- Note that these are all equivalent representations!

  ```scala
    //Equivalent representations of booleanGenerator
    val booleans1 = for x <- integerGenerator yield x > 0
    val booleans2 = integerGenerator.map(x => x > 0)
    val booleans3 = new Generator[Boolean]:
      def generate() = ((x: Int) => x > 0)(integerGenerator.generate())
    val booleans4 = new Generator[Boolean]:
      def generate() = integerGenerator.generate() > 0

    //Equivalent representations of pairGenerator
    def pairs1[T, U](t: Generator[T], u: Generator[U]): Generator[(T, U)] = {
      t.flatMap(
        x => u.map(
          y => (x, y)
        )
      )
    }

    def pairs2[T, U](t: Generator[T], u: Generator[U]): Generator[(T, U)] = {
      t.flatMap(
        x => new Generator[(T, U)] { 
          def generate() = (x, u.generate()) 
        }
      )
    }

    def pairs3[T, U](t: Generator[T], u: Generator[U]): Generator[(T, U)] = {
      new Generator[(T, U)]:
        def generate() = {
          (
            new Generator[(T, U)]: 
              def generate() = (t.generate(), u.generate())
          ).generate()
      }
    }

    def pairs4[T, U](t: Generator[T], u: Generator[U]): Generator[(T, U)] = {
      new Generator[(T, U)]:
        def generate() = (t.generate(), u.generate())
    }
  ```

### Exercise: Writing our own `List` generator

- We now know that a generator should implement the `map` and `flatMap` methods to enable us to use the `for` syntactic sugar

- This `for` syntax makes it super easy to write new `Generator`s, giving us very fine grained control to build up new functionality
  ```scala
  def single[T](x: T): Generator[T] = new Generator[T]:
    def generate() = x
  
  def range(lo: Int, hi: Int): Generator[Int] =
    for 
      x <- integers 
    yield 
      lo + x.abs % (hi - lo)
  
  def oneOf[T](xs: T*): Generator[T] =
    for 
      idx <- range(0, xs.length) 
    yield 
      xs(idx)
  ```

- Let's now try writing a generator for a `List[Int]`
  - This generates anything from a random list, to a list of arbitrary random length

  - See `4-slides.scala`

### Exercise: Write our own `Tree` generator

- See `4-slides.scala`

### Exercise: Using `Generator` in random testing

- One application of the `Generator` object is in generating test cases

- For example:
  ```scala
    def test[T](g: Generator[T], numTimes: Int = 100)(test: T => Boolean): Unit =
      for i <- 0 until numTimes do
        val value = g.generate()
        assert(test(value), s”test failed for $value”)
      println(s”passed $numTimes tests”)
  ```

- Example usage. Does the property always hold? Obviously not, because list can be empty

  ```scala
    test(pairs(lists, lists)) {
      (xs, ys) => (xs ++ ys).length > xs.length
    }
  ```

## Monads

- We looked at some data structures with `map` and `flatMap` defined (e.g. `Generator`). In fact, this is an entire class of objects known as **monads**

- In this section, we'll look at a rigorous definition of what monads are

### What are monads?

- Let `M` be some Monad. 

- Then a Monad of type T is any type `M[T]` with 2 operations; `unit` and `flatMap`, whereby

  ```scala
    extension[T, U](m: M[T]) 
      def flatMap(f: T => M[U]): M[U]
    
    def unit[T](x: T): M[T] 
  ```

- For example:
  - `List` is a monad
    ```scala
      val l: List[Int] = List(1,2,3)
      l.flatMap(x => List(x)) // will produce List[Int], when f is of type Int => List[Int]
      unit(1) = List(1) //define unit(x: Int): List[Int] = ???
    ```
  - `Set` is a monad
    ```scala
      val s: Set[Int] = Set(1,2,3)
      s.flatMap(x => Set(x)) // will produce Set[Int], when f is of type Int => Set[Int]
      unit(1) = Set(1) //define unit(x: Int): List[Int] = ???
    ```

- You may notice that `map` is not a required method. Why?
  - Because it turns out that `map` can be defined in terms of `flatMap` and `unit`. Think of `map` as a special case of `flatMap`
  ```scala
    type monad[T] = M[T]
    
    def flatMapFunc[T,U](x: T): M[U] //changes type T to U, and wraps it in monad M
    def mapFunc[T,U](x: T): U //changes type T to U

    monad.map(mapFunc) //produces M[U]
    monad.flatMap(x => unit(flatMapFunc(x))) //flatMapFunc(x) produces M[U], unit(.) produces M[M[U]], flatMap then flattens into M[U]. Therefore, equivalent
  ```

### What qualifies something as a monad?

- A monad has to follow 3 laws (i) Associativity, (ii) Left Unit, and (iii) Right Unit
  1. Associativity
    - Generally: `m.flatMap(f).flatMap(g) == m.flatMap(f(x).flatMap(g))`
    - e.g. `List(1,2,3).flatMap(x => List(x*2)).flatMap(x => List(x*2)) == List(1,2,3).flatMap(x => List(x*2).flatMap(y => List(y*2)))`

  2. Left Unit
    - Generally: `unit(x).flatMap(f) == f(x)`
    - e.g.
      ```scala
        def f(x: Int): Int = List(x * 2)
        unit(1).flatMap(x => f(x)) //unit(1) gives List(1). List(1).flatMap(x => List(x*2)) becomes List(2)
        f(1) //gives us List(2)
      ```

  3. Right Unit
    - Generally: `m.flatMap(unit) == m`
    - e.g.
      ```scala
        val m = List(1)
        m.flatMap(x => unit(x)) //gives us flatMap of List(List(1)), which is List(1), which is equal to m
      ```

## Exceptional Monads

- To write exceptions in Scala, extend the `Exceptions` base class
  ```scala
    class BadInput(msg: String) extends Exception(msg)
    throw new BadInput(”missing data”)
  ```

- You have a couple of options when it comes to try...catch... pattern in Scala
  - Rely on the usual try/catch pattern, which throws an error on failure
    ```scala
      def testFunc() = {
        try {
          something()
        } 
      }
    ```

  - Use `scala.util.Try`, which lets you see an exception as a "normal" result, and so doesn't break your program

- See `6-slides.scala`